In [1]:
import collections
import copy
import datetime
import itertools
import sys

import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

from cibblbibbl import bbcode

In [2]:
show_Ids = True
G = cibblbibbl.CIBBL
Achievement = cibblbibbl.achievement.Achievement
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay
Tournament = cibblbibbl.tournament.handler.default.Tournament
NormalPlayer = cibblbibbl.player.NormalPlayer
StarPlayer = cibblbibbl.player.StarPlayer

In [3]:
def bbcode_section(s):
  return bbcode.size(bbcode.b(bbcode.i(s)), 12)

In [4]:
G.init()

In [5]:
S = max(G.seasons)
print(S)

Season('cibbl', 7, 1)


In [6]:
T = G.tournaments["43150"]

In [7]:
T.rpts

{'W': 2, 'D': 1, 'B': 2}

In [6]:
S = S1

NameError: name 'S1' is not defined

In [8]:
S1 = S
S0 = S1.prev
print(f'Y{S1.year.nr} {S1.name}')

Y6 Autumn


In [6]:
S1 = S.prev
S0 = S1.prev
print(f'Y{S1.year.nr} {S1.name}')

Y6 Autumn


In [12]:
g1 = S1.gold_partner_teams()
s1 = S1.silver_partner_teams()
g0 = S0.gold_partner_teams()
s0 = S0.silver_partner_teams()

In [18]:
pdata = {
    "gold": {
	"decaymul": [
        1,
        0
	],
	"description": "Awarded to a team which is a consecutive CIBBL member for the last 4 CIBBL Years (16 Seasons). Replaces Silver partnership. Worth 50 Prestige Points to the team as long as the team continously participates in every CIBBL events. Awarded since Year 5 Summer.",
	"logo_url": "/i/597778",
	"name": "CabalVision Gold Partner",
    "prestige": 0,
    "small_logo_url": "/i/597782",
	"stackmul": [
		1,
		0
	],
	"start_tournamentId": "46687",
	"typename": "Achievement",
	"value": 50
},
    "silver": {
	"decaymul": [
        1,
        0
	],
	"description": "Awarded to a team which is a consecutive CIBBL member for the last 2 CIBBL Years (8 Seasons). Worth 25 Prestige Points to the team as long as the team continously participates in every CIBBL events or until it becomes a Gold Partner. Awarded since Year 5 Summer.",
	"logo_url": "/i/597779",
	"name": "CabalVision Silver Partner",
    "prestige": 0,
    "small_logo_url": "/i/597781",
	"stackmul": [
		1,
		0
	],
	"start_tournamentId": "46687",
	"typename": "Achievement",
	"value": 25
}
}

In [22]:
newteams = {
    "gold": g1-g0,
    "silver": s1-s0,
}
ended = {
    "gold": g0-g1,
    "silver": s0-s1-newteams["gold"],
}

In [23]:
ended

{'gold': {Team(854678), Team(876774)},
 'silver': {Team(897376), Team(900747), Team(914538), Team(916031)}}

In [ ]:
parts = []

Te = None
parts.append("\n")
parts.append(bbcode.center(bbcode.img("/i/552701")))
parts.append("\n")
parts.append("\n")
parts.append(bbcode_section("New Partnerships") + "\n")
parts.append(bbcode.hr() + "\n")
parts.append("[block=center]")

parts1 = []
for name, teams in newteams.items():
  if not teams:
    continue
  logo_url = pdata[name].get("logo_url")
  if logo_url:
    parts1.append(bbcode.img(logo_url) + "\n")
  parts1.append(bbcode.b(bbcode.i(pdata[name]["name"])) + "\n")
  parts1.append("\n")
  descr = bbcode.i(pdata[name]["description"])
  parts1.append(
      "[block=automargin width=67%]"
      + descr
      + "[/block]"
  )
  parts1.append("\n")
  for Te in sorted(teams, key=lambda Te: Te.name):
      parts1.append(bbcode.size(bbcode.team(Te), 12) + "\n")
  parts1.append("\n")
  parts1.append("\n")

snew = "".join(parts1).rstrip("\n")
parts.append(snew)

parts.append("[/block]")
anyended = {Te for sTe in ended.values() for Te in sTe}
if anyended:
    if Te:
        parts.append("\n")
        parts.append("\n")
    parts.append(bbcode_section("Partnership Ended") + "\n")
    parts.append(bbcode.hr() + "\n")
    parts.append("[block=center]")
    parts2 = []
    for name, sTe in ended.items():
        for Te in sorted(sTe, key=lambda Te: Te.name):
            parts2.append(bbcode.size(bbcode.team(Te) + f' (-{pdata[name]["value"]} Prestige Points)', 12) + "\n")
    sout = "".join(parts2).rstrip("\n")
    parts.append(sout)
    parts.append("[/block]")

In [25]:
print("".join(parts))


[block=center][img]/i/552701[/img][/block]

[size=12][b][i]New Partnerships[/i][/b][/size]
[block=automargin blackborder width=100%][/block]
[block=center][img]/i/597778[/img]
[b][i]CabalVision Gold Partner[/i][/b]

[block=automargin width=67%][i]Awarded to a team which is a consecutive CIBBL member for the last 4 CIBBL Years (16 Seasons). Replaces Silver partnership. Worth 50 Prestige Points to the team as long as the team continously participates in every CIBBL events. Awarded since Year 5 Summer.[/i][/block]
[size=12][url=/p/team?team_id=891260]Library of Indra[/url][/size]


[img]/i/597779[/img]
[b][i]CabalVision Silver Partner[/i][/b]

[block=automargin width=67%][i]Awarded to a team which is a consecutive CIBBL member for the last 2 CIBBL Years (8 Seasons). Worth 25 Prestige Points to the team as long as the team continously participates in every CIBBL events or until it becomes a Gold Partner. Awarded since Year 5 Summer.[/i][/block]
[size=12][url=/p/team?team_id=925779]Divine 